# Exercise 1 - Localize objects

### Step 0 - Imports && Setup

### Step 1 - Feature Detection && Description

#### SIFT

#### SURF  

#### ORB

#### BRISK

### Step 2 - Feature Matching

#### Brute-Force

#### FLANN

#### Step 3 - Correspondence rejection

#### Without RANSAC

#### With RANSAC

### Step 4 - Drawing the Polygon